In [1]:
from cellpose import models, io
from cellpose.io import *
from collections import defaultdict
import geopandas
import glob
import imagej
from jpype import JArray, JInt
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import os
import re
import pandas
from pandas import DataFrame
from pathlib import Path
import scyjava
import seaborn
import shutil
import tkinter as tk
from tkinter import filedialog
from PIL import Image

In [2]:
scyjava.config.add_option('-Xmx30g')
start_dir = os.getcwd()
#ij = imagej.init('sc.fiji:fiji', mode='interactive')
ij = imagej.init('/home/saka/sw/local/fiji/2023', mode='interactive')
## Something about this init() function changes the current working directory.
ij.getApp().getInfo(True)
ij.ui().showUI()
os.chdir(start_dir)
ij.getVersion() #This is to make sure ImageJ/Fiji opened properly. In case of error, it could display '2.9.0/inactive' instead of the full version number

'2.9.0/1.54e'

Unexpected exception formatting exception. Falling back to standard exception
ZeissCZIReader initializing /home/saka/Documents/Lab_stuff/confocal/exp2/Experiment-1647-Split Scenes-01.czi
Experiment-1647-Split Scenes-01.czi


In [3]:
showPolygonRoi = scyjava.jimport('ij.gui.PolygonRoi')
Overlay = scyjava.jimport('ij.gui.Overlay')
Regions = scyjava.jimport('net.imglib2.roi.Regions')
LabelRegions = scyjava.jimport('net.imglib2.roi.labeling.LabelRegions')
ZProjector = scyjava.jimport('ij.plugin.ZProjector')()
Duplicator = scyjava.jimport('ij.plugin.Duplicator')()
TrackMate = scyjava.jimport('fiji.plugin.trackmate.TrackMate')
ov = Overlay()
rm = ij.RoiManager.getRoiManager()

In [4]:
directory_path = '/home/saka/Documents/Lab_stuff/confocal/exp2'
#directory_path = '/home/saka/Documents/Zhihui/timelapse/images'
os.chdir(directory_path)

In [7]:
root = tk.Tk()
root.withdraw()
filepath = filedialog.askopenfilename()

In [8]:
image = ij.io().open(filepath)
#imp = ij.py.to_imageplus(image)

[java.lang.Enum.toString] ZeissCZIReader initializing /home/saka/Documents/Lab_stuff/confocal/exp2/Experiment-1647-Split Scenes-01.czi
[WARN] Unknown DetectorType value 'GaAsP-PMT' will be stored as "Other"
[java.lang.Enum.toString] [WARN] Unknown DetectorType value 'GaAsP-PMT' will be stored as "Other"
[WARN] Unknown DetectorType value 'GaAsP-PMT' will be stored as "Other"
[java.lang.Enum.toString] [WARN] Unknown DetectorType value 'GaAsP-PMT' will be stored as "Other"
[WARN] Unknown DetectorType value 'GaAsP-PMT' will be stored as "Other"
[java.lang.Enum.toString] [WARN] Unknown DetectorType value 'GaAsP-PMT' will be stored as "Other"
[WARN] Unknown DetectorType value 'Multialkali-PMT' will be stored as "Other"
[java.lang.Enum.toString] [WARN] Unknown DetectorType value 'Multialkali-PMT' will be stored as "Other"


In [20]:
wanted_channel = 2
channel = image[:, :, wanted_channel, :, :]
imp = ij.py.to_imageplus(channel)
imp.setDimensions(1, 11, 121)
ij.ui().show(imp)
f_name = os.path.basename(filepath)
f_name = os.path.splitext(f_name)[0] + "_LV.tif"
format = f'Tiff'
result_path = f"{directory_path}/{f_name}"
set_string = f'saveAs("{format}", "{result_path}")'
ij.py.run_macro(set_string)
ij.py.run_macro("close();")

<java object 'org.scijava.script.ScriptModule'>

In [18]:
result = os.path.basename(filepath)
print(result)

In [ ]:
testing = pandas.read_csv('nearest.csv')
for row_number in range(len(testing)):
    single_row = testing.iloc[row_number]
    row_cellids = single_row.child_cell_ids
    row_cleaned = row_cellids.strip('[').strip(']')
    row_array = row_cleaned.split(', ')
    for cell in row_array:
        print(f"This row has cell: {cell}")
        cell_index = int(cell)
        roi = rm.select(cell_index)

In [ ]:
for i in prout:
    roi = rm.select(i)
    duplicator_result_f"{"i"}" = Duplicator.run(imp, 1, 1, 1, 11, 1, 1)
    shown = ij.ui().show(duplicator_result)

In [ ]:
macro = """
title = getTitle();
a =
newArray(17,74,132,185,240,295,347,397,440,488,536,586,645,
697,750,804,856,645,697,750,804,856,909,958,1008,1059,1111,
1160,1210,1260,1307,1352,1395,1441,1486,1533,1580,1628,1670,
1716,1756,1800,1849,1897,1942,1986,2027,2072,2119,2164,2205,
2248,2286,2334,2375,2412,2456,2498,2538,2576,2613,2652
);
for ( i=0; i<a.length; i++ ) {
    roiManager( "Select", a[i]);
    Stack.getPosition(channel, slice, frame);
	run("Duplicate...", "duplicate frames="+frame);
    selectWindow(title);
}

selectWindow(title);
close();
run("Concatenate...", "all_open open");
"""

results = ij.py.run_macro(macro)

In [21]:
#so to fix the virtual stack format, it is required to duplicate it to work from the virtual stack processing menu that is not working. 
pretreatment = """
run("Duplicate...", "duplicate");
run("Smooth", "stack");
run("Smooth", "stack");
setAutoThreshold("Default dark no-reset");
run("Threshold...");
setThreshold(5, 255);
setOption("BlackBackground", true);
run("Convert to Mask", "black");
run("Set Measurements...", "area centroid stack redirect=None decimal=3");
run("Analyze Particles...", "display add stack");
"""
smoothed = ij.py.run_macro(pretreatment)

#saving if the result table
measurements = ij.ResultsTable.getResultsTable() 
measurements_table = ij.convert().convert(measurements, scyjava.jimport('org.scijava.table.Table')) 
table = ij.py.from_java(measurements_table) 
results = os.path.basename(filepath)
results = os.path.splitext(results)[0]
output_path = Path(f"{directory_path}/{results}.csv")
table.to_csv(output_path)
ij.py.run_macro(""" 
title = Table.title();
selectWindow(title);
run("Close");
""")

#saving the ROIs
f_name = os.path.basename(filepath)
f_name = os.path.splitext(f_name)[0]
ij.py.run_macro("roiManager('Select All');")
rm.runCommand("Save", f"{directory_path}/" + f"{f_name}.zip")
rm.runCommand("Delete")

True

In [17]:
macro = """
title = getTitle();
a =
newArray(10, 67, 123, 175, 232, 286, 339, 390, 435, 482, 528, 580, 635, 690, 743, 
797, 851, 903, 952, 1002, 1052, 1104, 1154, 1205, 1256, 1303, 1348, 1392, 1437, 
1484, 1528, 1577, 1624, 1667, 1713, 1753, 1796, 1846, 1894, 1938, 1983, 2024, 2070, 
2116, 2160, 2200, 2245, 2284, 2330, 2373, 2411, 2454, 2495, 2537, 2575, 2612, 2651, 2690, 2730, 2771, 2806, 2841,
2875, 2909, 2947, 2983, 3024, 3064, 3106, 3144, 3180, 3221, 3267, 3309, 3341, 3374, 3410, 3442, 3481, 3516, 3549, 
3578, 3612, 3647, 3680, 3713, 3750, 3780, 3817, 3850, 3882, 3913, 3943, 3973, 4003, 4033, 4062, 4090, 4119, 4149, 
4180, 4209, 4239, 4262, 4293, 4322, 4356, 4387, 4422, 4450, 4479, 4510, 4538, 4567, 4596, 4624, 4652, 4677, 4704);
for ( i=0; i<a.length; i++ ) {
    roiManager( "Select", a[i]);
    Stack.getPosition(channel, slice, frame);
	run("Duplicate...", "duplicate frames="+frame);
    selectWindow(title);
}

selectWindow(title);
close();
run("Concatenate...", "all_open open");
"""

results = ij.py.run_macro(macro)